In [8]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import glob

In [2]:
x, y, width, height = 450, 30, 1024, 768

num_folders = 8
images_per_folder = 2354
base_output_dir = "D:/dataset/train/data_train"
for i in range(num_folders):
    folder_path = f"{base_output_dir}/data_{i+1}"
    os.makedirs(folder_path, exist_ok=True)

for idx, filename in enumerate(glob.glob("D:/dataset/data/images/*.jpg")):
    filename = filename.replace(os.sep, "/")
    depth_filename = filename.replace("images", "scans")
    depth_filename = depth_filename.replace("frame", "scan")
    file = (filename.split("/")[-1]).rsplit(".")[0]
    file = file.replace("frame_", "")
    
    if not os.path.isfile(depth_filename):
        print(filename, "scan not found")
        continue
    
    # Specifying a folder based on the index
    folder_idx = idx // images_per_folder
    if folder_idx >= num_folders:
        print("Warning: More images than expected. Adjust the number of folders or images_per_folder.")
        break
    
    output_folder = f"{base_output_dir}/data_{folder_idx+1}"
    
    image = cv2.imread(filename)
    depth = cv2.imread(depth_filename)
    mask = cv2.inRange(depth[y:y+height, x:x+width], (0), (0))
    dst = cv2.inpaint(depth[y:y+height, x:x+width], mask, 3, cv2.INPAINT_TELEA)
    
    cv2.imwrite(f"{output_folder}/{file}.jpg", image[y:y+height, x:x+width])
    cv2.imwrite(f"{output_folder}/{file}.png", dst)

In [5]:
%run datasets.ipynb

In [6]:
write_csv("D:/dataset/train/data_train.csv", "/*/*.jpg", (".jpg", ".png"))

In [9]:
for filename in glob.glob("D:/dataset/train/data_train/*/*"):
    image = cv2.imread(filename)
    h, w, _ = image.shape
    if h != 240 or w != 320:
        image = cv2.resize(image, (320, 240))
        cv2.imwrite(filename, image)

In [ ]:
mind = np.inf
maxd = -np.inf

for filename in glob.glob("D:/dataset/train/data_train/*/*.png"):
    image = cv2.imread(filename)
    mind = min(mind, np.min(image))
    maxd = max(maxd, np.max(image))

print(mind, maxd)